In [1]:
from collections import Counter
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

plt.rcParams['font.size'] = 12

In [2]:
with open('results/cache/result_run_9268772.json', 'r') as f:
    pred = json.load(f)
pred[0]

{'answer': 'down', 'question_id': 262148000}

In [3]:
with open('data/vqa/raw/v2_OpenEnded_mscoco_val2014_questions.json') as f:
    ques = json.load(f)['questions']
ques[0]

{'image_id': 262148,
 'question': 'Where is he looking?',
 'question_id': 262148000}

In [4]:
with open('data/vqa/raw/v2_mscoco_val2014_annotations.json') as f:
    anno = json.load(f)['annotations']
anno[0]

{'question_type': 'none of the above',
 'multiple_choice_answer': 'down',
 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 262148,
 'answer_type': 'other',
 'question_id': 262148000}

In [5]:
pred_df = pd.DataFrame(pred)
ques_df = pd.DataFrame(ques)
anno_df = pd.DataFrame(anno)

df = pred_df.merge(ques_df, how='inner').merge(anno_df, how='inner')
df = df.reindex(columns=['image_id','question_id', 'question_type', 'question',
                         'answer_type', 'multiple_choice_answer', 'answers', 'answer'],
                copy=False)
# df.rename(columns=lambda col: 'prediction' if col == 'answer' else col, inplace=True)

del pred, ques, anno, pred_df, ques_df, anno_df

df.head()

,image_id,question_id,question_type,question,answer_type,multiple_choice_answer,answers,answer
0,262148,262148000,none of the above,Where is he looking?,other,down,"[{'answer': 'down', 'answer_confidence': 'yes'...",down
1,262148,262148001,what are the,What are the people in the background doing?,other,watching,"[{'answer': 'spectating', 'answer_confidence':...",watching
2,262148,262148002,what is,What is he on top of?,other,picnic table,"[{'answer': 'table', 'answer_confidence': 'yes...",picnic table
3,393225,393225000,what,What website copyrighted the picture?,other,foodiebakercom,"[{'answer': 'foodiebakercom', 'answer_confiden...",flickr
4,393225,393225001,is this a,Is this a creamy soup?,yes/no,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",no


In [6]:
ans_set = df['answers'].map(lambda ans: set(map(lambda a: a['answer'], ans)))
wrong_idx = [a not in s for a,s in zip(df['answer'], ans_set)]

wrong = df[wrong_idx]
wrong.head()

,image_id,question_id,question_type,question,answer_type,multiple_choice_answer,answers,answer
3,393225,393225000,what,What website copyrighted the picture?,other,foodiebakercom,"[{'answer': 'foodiebakercom', 'answer_confiden...",flickr
21,262162,262162003,how many,How many pictures are there?,number,7,"[{'answer': '7', 'answer_confidence': 'yes', '...",8
40,262162,262162022,how many,How many frames are on the wall?,number,7,"[{'answer': '7', 'answer_confidence': 'yes', '...",6
44,262162,262162026,what,What animal print does that chair resemble?,other,none,"[{'answer': 'panther', 'answer_confidence': 'y...",0
46,262162,262162028,what is the,What is the size of the bed?,other,queen,"[{'answer': 'queen', 'answer_confidence': 'yes...",twin
